# DSS Example: Maximize Harvesting via optimization

This notebook is an example implementation of a notebook-based decision-support system (DSS) for evaluating the net system-level carbon emissions impact of an maximize harvesting_ type project. 

## DSS implementation notes

At its core this DSS prototype uses ws3 and libcbm_py to do most of the heavy lifting in terms of simulating forest growth, simulating harvesting disturbances, and simulating forest ecosystem carbon stocks and fluxes. ws3 includes built-in functions that automate most of process of compiling CBM input data, importing this input data into CBM, running CBM, and extracting CBM output data in tablular format. We layer onto this some custom functions to simulate multiple decaying pools of harvested wood products (HWP), and displacement effects for these. 

This DSS assumes we are using the standard "alternative scenario net of baseline scenario" approach to defining net carbon emissions, so we include functions that automate the process of pushing a pair of (baseline, alternative) scenarios through the simulation pipeline, collecting the output from each scenario, calculating the difference across scenarios (alternative minus baseline) and displaying this contrast as a function of time.

We include a few interactive widgets to show how these can be used to implement a more user-friendly interface for the DSS while keeping the flexibility and low overhead cost of using a Jupyter notebook environment to implement the DSS. 

Note that most of the complex code that defines the behaviour of the DSS and underlying simulation engines is implemented in a companion `util_opt` Python module. The functions we need from the `util_opt` module are imported at the top of this notebook. 


## Description of case

The study area for this case is a part of timber supply area (TSA) 04 (Cassier) in British Columbia around a mining site with size of 404,590 ha. In this hypothetical case we are assuming that harvesting will be planned such that a _maximizing harvesting_ of merchantable timber volume is extracted from the study area (over an arbitrary 100-year simulation horizon, althought the system can easily be parametrized to simulate shorter or longer horizons). We assume that the harvesting will begin unless someone "buys out" and holds the harvesting rights without exploiting them (effectively implementing a "no harvesting" policy going forward). In this situation, the "harvesting" policy represents the baseline scenario and the "no harvesting" policy is the alternative scenario.  

In [ ]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import ws3.forest
from util_opt import results_scenarios, run_scenario
import time
import gurobipy as grb

In [ ]:
base_year = 2020
horizon = 10
period_length = 10
max_age = 1000
n_steps = 100
tvy_name = 'totvol'
case_study =  'redchrs'
output_csv_path='./outputs/csv'
output_pdf_path='./outputs/pdf'

Create some interactive widgets to control four model parameters. The widgets have reasonable default values defined, but DSS users can override these default values. There are many (thousands!) of other model parameters that could modified to tweak model behaviour. These are complex models, and in the end the quality of output data is limited by the quality of input parameter choices (many of which require expert judgement).  

In [ ]:
hwp_pool_effect = widgets.FloatSlider(min=0, max=1, step=1, description='HWP_effect', value=1)

display(hwp_pool_effect)

In [ ]:
hwp_pool_effect_value = hwp_pool_effect.value

In [ ]:
# Here we indicate whether the harvested pool products turn into emissions immediately instead of turning into wood products.
if hwp_pool_effect_value == 0:
    release_immediately = widgets.FloatSlider(min=0, max=1, step=1, description='Release immediately', value=1)
    display(release_immediately)

In [ ]:
if hwp_pool_effect_value == 0:
    release_immediately_value = release_immediately.value
else:  
    release_immediately_value = 0

In [ ]:
if hwp_pool_effect_value != 0:
    displacement_effect = widgets.FloatSlider(min=0, max=1, step=1, description='Displacement', value=1)
    display(displacement_effect)

In [ ]:
if hwp_pool_effect_value != 0:
    displacement_effect = displacement_effect.value
else:  
    displacement_effect = 0

In [ ]:
if displacement_effect == 0:
   clt_percentage = 0
   credibility = 0
else:
    clt_percentage = widgets.FloatSlider(min=0, max=1, step=0.1, description='CLT_percent:', value=0.5)
    credibility = widgets.FloatSlider(min=0, max=1, step=0.1, description='Credibility:', value=1)
    display(clt_percentage, credibility)

In [ ]:
max_harvest = widgets.FloatSlider(min=0, max=1, step=0.05, description='Max_harvest:', value=0.1)
budget_input = widgets.FloatText(
    value=10000000,
    description='Budget:',
    disabled=False)
display(max_harvest, budget_input)

In [ ]:
def select_scenario(scenario_name):
    print(f"Selected scenario: {scenario_name}")
scenario_dropdown = widgets.Dropdown(
    options=['no_cons', 'bau_redchrs', 'redchrs_gs_hv_ha_100', 'redchrs_gs_hv_ha_90'],
    value='redchrs_gs_hv_ha_100',
    description='Scenario:'
)
display(scenario_dropdown)
scenario_dropdown.observe(lambda change: select_scenario(change['new']), names='value')

In [ ]:
def select_obj(obj_mode):
    print(f"Selected scenario: {obj_mode}")
obj_dropdown = widgets.Dropdown(
    options=['max_hv', 'min_ha'],
    value='max_hv',
    description='Objective:'
)
display(obj_dropdown)
obj_dropdown.observe(lambda change: select_obj(change['new']), names='value')

In [ ]:
if displacement_effect != 0:
    clt_percentage = clt_percentage.value
    credibility = credibility.value
max_harvest = max_harvest.value
budget_input = budget_input.value
scenario_name = scenario_dropdown.value
obj_mode = obj_dropdown.value

In [ ]:
fm = ws3.forest.ForestModel(model_name='redchris',
                            model_path='./data/woodstock_model_files_redchris',
                            base_year=base_year,
                            horizon=horizon,
                            period_length=period_length,
                            max_age=max_age)
fm.import_landscape_section()
fm.import_areas_section()
fm.import_yields_section()
fm.import_actions_section()
fm.import_transitions_section()
fm.initialize_areas()
fm.add_null_action()
fm.reset_actions()

Run the simulations and compile output data into a "dashboard" of graphs. The grid of four subfigures shows carbon stocks and emissions for both baseline and alternative scenarios. The figure on the bottom shows the contrast (difference) between net emissions for alternative and baseline scenarios. The contrast is a negative number (in most time steps), which means that according to this simulation the alternative (no harvesting) scenario generally has lower emissions behaviour than the baseline (high even-flow harvesting) scenario. 

However, close inspection of the detailed scenario-wise emissions graphs reveals that the absolute magnitude of the "net emission" indictator for each of the scenarios is relatively small compared to the absolute magnitude of "ecosystem decay" and "gross growth" emissions (which dominate the net emissions calculation, and come close to cancelling each other out completely). Furthermore, side-by-side visual inspection of the relatively small-magnitude "net emissions" profiles for each scenario shows that they are _very_ similar.

So, althrough this model is calculating a net negative impact of the "no harvesting" scenario relative to the "with harvesting" scenario, the magnitude of this impact is small enough (compared to the magnitude of the constituent carbon emission components that are summed to calculate the net emission indicator) to trigger a "materiality" criterion challenge. Materiality is further challenged when one considers the relatively low confidence (i.e., high uncertainty) associated with some of the key input assumptions used in this deterministic analysis). 

As a further example of how the flexibility of this type of notebook-based DSS implementation, we automate the process of calculating the avoided emissions for the first 25 years of the simulation horizon (i.e., the area under the leftmost part of the curve in the bottom figure in the dashboard), and dividing this number into a hypothetical "budget" for buying out the harvesting rights, yielding a "unit cost of avoided emissions" (in $ per tCO2e). This is the same sort of calculation (minus a full-blown techno-economic analysis to back up the budget value in the numerator) used by economists to compile marginal abatement cost curve (MACC) figures. For example, the DSS prototype as it is implemented could be re-run multiple times to find (through trial and error) the maximum total price one would want to bid to buy out the harvesting rights on this piece of land, given a maximum "unit cost of avoided emissions" price we were willing to pay (above which price maybe it would be more economically efficient to invest our "carbon money" elsewhere, or just purchase ready-baked carbon offset credits from one of several retail voluntary carbon offset markets).

In [ ]:
harvest_acode='harvest'
fm.actions[harvest_acode].is_harvest = True

In [ ]:
fm.reset()

In [ ]:
# schedule = run_scenario(fm, obj_mode, scenario_name)
# schedule

In [ ]:
# , 'redchrs_gs_hv_ha_100', 'redchrs_gs_hv_ha_90'
scenario_names = ['no_cons', 'bau_redchrs']
for scenario_name in scenario_names:
    Start = time.time()  # Start timing for the current scenario
    print(f"Running for {scenario_name}...")    
    results_scenarios(fm, 
                  clt_percentage, 
                  credibility, 
                  budget_input, 
                  n_steps, 
                  max_harvest, 
                  scenario_name, 
                  displacement_effect, 
                  hwp_pool_effect_value, 
                  release_immediately_value, 
                  case_study, 
                  obj_mode, 
                  output_csv_path, 
                  output_pdf_path, 
                  pickle_output_base=True, 
                  pickle_output_alter=True)
    # Print the time taken for the current scenario
    print(f"It took {round((time.time() - Start) / 60, 1)} minutes to run {scenario_name}.")